In [1]:
using Revise

[FProfile.jl](https://github.com/cstjean/FProfile.jl) provides an alternative interface for [Julia's sampling profiler](https://docs.julialang.org/en/latest/manual/profile/). Please read the introduction of that document before proceeding if you've never used `@profile`.

# Profiling

You can build a profile by calling `@fprofile(code, delay=0.001, n_samples=1000000)`:

In [2]:
using FProfile, Calculus

pd = @fprofile second_derivative(sin, 1.0)

INFO: Recompiling stale cache file /Users/cedric/.julia/lib/v0.6/FProfile.ji for module FProfile.


ProfileData(42 backtraces)

`@fprofile(N, ...)` is shorthand for `@fprofile(for _ in 1:N ... end)`:

In [3]:
pd = @fprofile 1000000 second_derivative(sin, 1.0)

ProfileData(967 backtraces)

Do not forget that Julia compiles code the first time a function is run; if you do not want to measure compilation time, execute your code once before profiling.

# Flat view

FProfile's `flat` report is a [dataframe](http://juliadata.github.io/DataFrames.jl/stable/man/getting_started/#Getting-Started-1). No particular knowledge of dataframes is necessary. I'll provide a few common operations below.

In [4]:
using DataFrames

df = flat(pd)
head(df, 15)   # show only the first 15 rows (the 15 rows with the highest count)

,count_pct,stackframe
1,99.9,"execute_request(::ZMQ.Socket, ::IJulia.Msg) at execute_request.jl:154"
2,99.9,anonymous at <missing>:?
3,99.9,macro expansion at FProfile.jl:52 [inlined]
4,99.9,(::IJulia.##14#17)() at task.jl:335
5,99.9,eventloop(::ZMQ.Socket) at eventloop.jl:8
6,99.9,macro expansion at profile.jl:23 [inlined]
7,99.9,"include_string(::Module, ::String, ::String) at Compat.jl:464"
8,99.9,macro expansion at FProfile.jl:67 [inlined]
9,99.9,"include_string(::String, ::String) at loading.jl:515"
10,99.07,"second_derivative(::Function, ::Float64) at derivative.jl:71"


The first column shows what fraction of backtraces (in %) go through the `method at file:line_number` in the `stackframe` column. It's the same quantity as in `Base.Profile.print()`, except for recursive calls: if `f(1)` calls `f(0)`, that's 2 counts in Base's report, but only 1 count in FProfile.

You can select a subset of the dataframe by using one of the five accessors: `get_specialization, get_method, get_file, get_function` and `get_module`.

In [5]:
df[get_function.(df[:stackframe]) .== derivative, :]  # select the `derivative` rows

,count_pct,stackframe
1,50.67,"derivative(::Function, ::Symbol, ::Symbol) at derivative.jl:3"
2,0.52,"derivative(::Function, ::Symbol, ::Symbol) at derivative.jl:0"


It is common to focus optimization efforts on one or more modules at a time (... the ones you're developing). `flat(pd, MyModule)` filters out other modules and adds a useful column: `self_pct` measures how much `MyModule`-specific work is done on that line.

For instance, in the code below, while the `do_computation()` call takes a long time (it has a high `count_percent`), it merely calls another `Main` function, so it has a low `self_pct`. `sum_of_sin` has `self_pct = 100%` because while it calls `sum` and `sin`, those are defined in another module (`Base`), and counted as external to `Main`. 

`flat(pd, (Module1, Module2, ...))` is also accepted.

In [6]:
@noinline do_computation(n) = sum_of_sin(n)
@noinline sum_of_sin(n) = sum(sin, 1:n)
pd2 = @fprofile do_computation(10000000)
flat(pd2, Main)

,count_pct,self_pct,stackframe
1,100.0,0.0,do_computation(::Int64) at In[6]:1
2,100.0,100.0,sum_of_sin(::Int64) at In[6]:2


It pays to make sure that functions with a high `self_pct` are [well optimized](https://docs.julialang.org/en/latest/manual/performance-tips/).

Another way to reduce the level of detail is to aggregate by `:specialization, :method, :file, :function`, or `:module`.

In [7]:
df_by_method = flat(pd, combineby=:method)

,count_pct,method
1,99.9,missing_info() in FProfile.MissingInfo at /Users/cedric/.julia/v0.6/FProfile/src/FProfile.jl:25
2,99.9,eventloop(socket) in IJulia at /Users/cedric/.julia/v0.6/IJulia/src/eventloop.jl:2
3,99.9,(::IJulia.##14#17)() in IJulia at task.jl:335
4,99.9,"include_string(mod::Module, code::String, fname::String) in Compat at /Users/cedric/.julia/v0.6/Compat/src/Compat.jl:464"
5,99.9,"execute_request(socket, msg) in IJulia at /Users/cedric/.julia/v0.6/IJulia/src/execute_request.jl:115"
6,99.9,"include_string(txt::String, fname::String) in Base at loading.jl:515"
7,99.48,"second_derivative(f::Function, x::Number) in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/derivative.jl:71"
8,51.19,"derivative(f::Function, ftype::Symbol, dtype::Symbol) in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/derivative.jl:2"
9,19.03,"finite_difference_hessian(f::Function, g::Function, x::Number, dtype::Symbol) in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/finite_difference.jl:224"
10,16.65,"finite_difference(f::Function, x::T, dtype::Symbol) where T<:Number in Calculus at /Users/cedric/.julia/v0.6/Calculus/src/finite_difference.jl:48"


You can see the context (caller/called functions) around each of these rows by passing it to `tree`:

In [8]:
tree(pd, df_by_method, 9)   # show the context of the 9th row of `df_by_method`

 184 ...ulus/src/derivative.jl:71; second_derivative(::Function, ::F...
  180 .../finite_difference.jl:224; finite_difference_hessian(::Func...
   127 .../finite_difference.jl:55; finite_difference(::Calculus.##1...
   28  .../finite_difference.jl:27; finite_difference(::Calculus.##1...
   2   .../finite_difference.jl:48; finite_difference(::Calculus.##1...
   1   .../finite_difference.jl:0; finite_difference(::Calculus.##1...
   2   .../finite_difference.jl:54; finite_difference(::Calculus.##1...
  4   .../finite_difference.jl:0; finite_difference_hessian(::Func...


Other useful dataframe commands:

```julia
df[[:count_percent, :method]]   # select only those two columns
sort(df, cols=:self_pct, rev=true)  # sort by self_pct
showall(df)   # show the whole dataframe
```

See `?flat` for more options.

#### Comparing results

Pass two `ProfileData` objects to `flat` to compare them. The third column is simply the difference between the first two. Use `sort(df, cols=:count_pct_diff, rev=true)` to show the worst performance regressions.

In [9]:
pd2 = @fprofile 1000000 second_derivative(sin, 1.0)
flat(pd, pd2, combineby=:function)

,count_pct_1,count_pct_2,count_pct_diff,function
1,51.19,55.83,4.64,Calculus.derivative
2,19.03,17.43,-1.6,Calculus.finite_difference_hessian
3,16.65,15.1,-1.55,Calculus.finite_difference
4,99.48,99.59,0.11,Calculus.second_derivative
5,99.9,99.8,-0.1,IJulia.execute_request
6,99.9,99.8,-0.1,include_string
7,99.9,99.8,-0.1,FProfile.MissingInfo.missing_info
8,99.9,99.8,-0.1,IJulia.eventloop
9,99.9,99.8,-0.1,IJulia.#14


Of course, this is most useful when comparing different algorithms or commits (see [Revise.jl](https://github.com/timholy/Revise.jl)). The differences in the above table are  just noise.

# Tree view

FProfile's tree view looks the same as `Base.Profile.print(format=:tree)`. The numbers represent raw counts. (If some branches seem out of place, see [this issue](https://github.com/JuliaLang/julia/issues/9689))

In [10]:
tr = tree(pd)

 966 ./task.jl:335; (::IJulia.##14#17)()
  966 ...Julia/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
   966 ...rc/execute_request.jl:154; execute_request(::ZMQ.Socket, ::...
    966 ...Compat/src/Compat.jl:464; include_string(::Module, ::Stri...
     966 ./loading.jl:515; include_string(::String, ::String)
      966 ./<missing>:?; FProfile.MissingInfo.missing_info
       966 ...ile/src/FProfile.jl:52; FProfile.MissingInfo.missing_info
        966 ./profile.jl:23; FProfile.MissingInfo.missing_info
         966 ...ile/src/FProfile.jl:67; FProfile.MissingInfo.missing_info
          958 .../src/derivative.jl:71; second_derivative(::Function...
           180 ...ite_difference.jl:224; finite_difference_hessian(:...
            127 ...ite_difference.jl:55; finite_difference(::Calculu...
             35 ...ite_difference.jl:27; finite_difference(::Base.#s...
             77 ...ite_difference.jl:55; finite_difference(::Base.#s...
              59 ./math.jl:419; FProfile.MissingInfo.missi...
   

Like `flat` reports, trees can be aggregated by `:specialization, :method, :file, :function`, or `:module`:

In [11]:
tree(pd, combineby=:module)

 966 IJulia
  966 Compat
   966 Base
    966 FProfile.MissingInfo
     962 Calculus
      59 FProfile.MissingInfo
 1   Calculus


If you're only interested in a particular module/file/method/function, you can pass it to `tree`, along with an optional _neighborhood range_.

In [12]:
tr_deriv = tree(pd, second_derivative, -1:1)    # -1:1 = show one level of callers and one level of called functions

 962 ...rofile/src/FProfile.jl:67; FProfile.MissingInfo.missing_info
  958 ...lus/src/derivative.jl:71; second_derivative(::Function, ::F...
   180 .../finite_difference.jl:224; finite_difference_hessian(::Func...
   490 ...lus/src/derivative.jl:3; derivative(::Function, ::Symbol,...
   5   ...lus/src/derivative.jl:0; derivative(::Function, ::Symbol,...
   4   .../finite_difference.jl:0; finite_difference_hessian(::Func...
  4   ...lus/src/derivative.jl:0; second_derivative(::Function, ::F...


In [13]:
is_C_call

is_C_call (generic function with 1 method)

Trees are an indexable and filterable datastructure. Use the accessors (see above) and `is_inline/is_C_call` in your `filter` predicate.

In [14]:
tr_deriv[1,1]

958 ...ulus/src/derivative.jl:71; second_derivative(::Function, ::Fl...
 180 .../finite_difference.jl:224; finite_difference_hessian(::Funct...
 490 ...lus/src/derivative.jl:3; derivative(::Function, ::Symbol, ...
 5   ...lus/src/derivative.jl:0; derivative(::Function, ::Symbol, ...
 4   .../finite_difference.jl:0; finite_difference_hessian(::Funct...


# Backtraces

(if you want to build your own analysis)

The raw profiler data is available either through `Base.Profile.retrieve()`, or through `pd.data` and `pd.lidict`. However, you might find `FProfile.backtraces(::ProfileData)` more immediately useful. 

In [15]:
count, trace = backtraces(pd)[1]  # get the first unique backtrace
@show count                       # the number of times that trace occurs in the raw data
trace

count = 2


13-element Array{StackFrame,1}:
 (::IJulia.##14#17)() at task.jl:335                                                                  
 eventloop(::ZMQ.Socket) at eventloop.jl:8                                                            
 execute_request(::ZMQ.Socket, ::IJulia.Msg) at execute_request.jl:154                                
 include_string(::Module, ::String, ::String) at Compat.jl:464                                        
 include_string(::String, ::String) at loading.jl:515                                                 
 anonymous at <missing>:?                                                                             
 macro expansion at FProfile.jl:52 [inlined]                                                          
 macro expansion at profile.jl:23 [inlined]                                                           
 macro expansion at FProfile.jl:67 [inlined]                                                          
 second_derivative(::Function, ::Float64)

Use the `get_method, get_file, ...` functions on `StackFrame` objects (see above). `tree(pd::ProfileData)` is defined as `tree(backtraces(pd))`, and similarly for `flat`, so you can modify the backtraces and get a tree/flat view of the results.